In [1]:
import numpy as np
from networkx.algorithms.distance_measures import periphery, eccentricity
import random

# Multiple rumour sources

When we want to detect multiple rumour sources we need to adapt rumour centrality...

a few steps that need to happen:
- update simulation to use multiple rumour sources
- understand `Chen et al. - 2016 - Detecting Multiple Information Sources in Networks.pdf` to adapt their solution
- implemented their graph splitting
- implemented their rest of algorithm
- implemented their comparisons
- adapted rumour centrality to split graphs
- wrote evaluation

### Chen et al. 2016 Algorithm overview (for SRI)

1. Clustering (split graph into partitions)
2. Localization (find sources per partition)

**Clustering**
- select two infected nodes with max distance (are most likely in different clusters)
- iteratively select as many nodes as needed that have max distance to current selection
- group all other nodes based on min distance to selected cluster nodes

**Localization** for trees
- for each cluster compute infection radius: $r_s = \lfloor max_{a,b \in V_s}{\frac{d(a,b)}{2}} \rfloor$, i.e. the maximum distance divided by two of any two nodes in a cluster
- compute max infection radius $r_{max} = max_{s} {r_s}$
- construct a tree T from original tree with chosen cluster representatives as leaves
- for each cluster representative find node which is $r_{max}$ away from it and choose it as one source estimate

**Localization** for general graphs
- find jordan infection centers for each cluster


**Heuristic for Estimating S** (Number of clusters)
- Choose large number $\bar{S}$
- for all $1 \le k \le \bar{s}$ compute clusters and $w_k = r_{max}$
- Set estimate of S to: $arg\max_{1 \le k \le \bar{S} - 2} {w_k - w_{k+1} - (w_{k+1} - w_{k+2})}$ (since the dropoff when you have to little centers to the next one should be big)

**Proposed alternative approach** (Clustering)
- k-means cluster with random centroids + distance centrality to choose next iteration's cluster center
- also seems to work pretty well


In [83]:
from typing import List
import networkx


def get_cluster_reprs(g: networkx.Graph, number_clusters: int) -> List[int]:
    # select nodes that are the farthest away from each other (and are infected)
    peripheral_nodes = periphery(g)
    cluster_reprs = random.sample(peripheral_nodes, k=2)

    dist_x = single_source_shortest_path_length(g, cluster_reprs[0])
    dist_y = single_source_shortest_path_length(g, cluster_reprs[1])
    summed_dists = {k: dist_x.get(k, 0) + dist_y.get(k, 0) for k in (set(dist_x) & set(dist_y) - set(cluster_reprs))}
    for k in range(2, number_clusters):
        # select node which is farthest away from currently selected nodes
        # sum distances from each node from set?
        max_node = sorted(summed_dists.items(), key=lambda x: x[1], reverse=True)[0]
        print(f"got node {max_node[0]} with dist {max_node[1]} to cluster centers")
        cluster_reprs.append(max_node[0])
        dist_new = single_source_shortest_path_length(g, cluster_reprs[1])
        summed_dists = {k: dist_x.get(k, 0) + dist_y.get(k, 0) for k in (set(summed_dists) & set(dist_new) - set(cluster_reprs))}

    return cluster_reprs

In [98]:
from typing import List, Dict
import networkx


def assign_all_nodes_cluster(g: networkx.Graph, cluster_reprs: List[int]) -> Dict[int, int]:
    dists = [single_source_shortest_path_length(g, cluster_repr) for cluster_repr in cluster_reprs]
    cluster_assign = {}

    for node in g:
        min_dist_cluster = 0
        for idx in range(len(cluster_reprs)):
            if dists[idx][node] < dists[min_dist_cluster][node]:
                min_dist_cluster = idx

        cluster_assign[node] = min_dist_cluster

    return cluster_assign

In [55]:
from typing import List
import networkx

def cluster_graph(g: networkx.Graph, number_clusters: int) -> List[int]:
    reprs = get_cluster_reprs(g, number_clusters)
    assignm = assign_all_nodes_cluster(g, reprs)
    return list(assignm.values())

In [97]:
from typing import List, Dict


def get_induced_subgraph(adj_list: Dict[int, List[int]], subgraph_assignments: List[int]) -> List[Dict[int, List[int]]]:
    return [
        {
            cur_node: [next_node for next_node in adj_list[cur_node] if subgraph_assignments[next_node] == i]
                for cur_node in adj_list if subgraph_assignments[cur_node] == i
        } for i in set(subgraph_assignments)
    ]

In [101]:
from typing import Tuple
from rumor_centrality.rumor_detection import networkx_graph_to_adj_list
from rumor_centrality.rumor_detection import get_center_prediction

def multiple_rumor_source_prediction(g: networkx.Graph, number_clusters: int) -> Tuple[List[int], List[int]]:
    assignm = cluster_graph(g, number_clusters)
    adj_list = networkx_graph_to_adj_list(g)
    subgraphs = get_induced_subgraph(adj_list, assignm)
    subgraphs_rumor_centers = list(map(get_center_prediction, subgraphs))

    return subgraphs_rumor_centers, assignm

In [75]:
from rumor_centrality import graph_generator
from rumor_centrality.graph_visualization import plot_nx_graph

random_graph = graph_generator.scale_free(30).to_undirected()
plot_nx_graph(random_graph)

In [76]:
eccentricity(random_graph, v=3)

4

In [77]:
periphery(random_graph)

[19, 24, 26]

In [78]:
from networkx.algorithms.components import is_strongly_connected
from networkx.algorithms.shortest_paths.unweighted import single_source_shortest_path_length

In [79]:
sorted(single_source_shortest_path_length(random_graph, 16).items(), key=lambda x: x[1], reverse=True)

[(12, 4),
 (19, 4),
 (26, 4),
 (1, 3),
 (3, 3),
 (5, 3),
 (6, 3),
 (7, 3),
 (8, 3),
 (9, 3),
 (10, 3),
 (13, 3),
 (14, 3),
 (15, 3),
 (17, 3),
 (21, 3),
 (22, 3),
 (23, 3),
 (24, 3),
 (27, 3),
 (28, 3),
 (29, 3),
 (0, 2),
 (2, 2),
 (4, 2),
 (18, 2),
 (20, 2),
 (25, 2),
 (11, 1),
 (16, 0)]

## TODO:
- Build vis that shows clusters

In [80]:
a_graph = graph_generator.scale_free(300).to_undirected()
cluster_assignm = cluster_graph(a_graph, 6)
plot_nx_graph(a_graph, cluster_assignm, cluster_assignm)

got node 161 with dist 12 to cluster centers
got node 246 with dist 12 to cluster centers
got node 295 with dist 12 to cluster centers
got node 186 with dist 11 to cluster centers


In [81]:
cluster_reprs = get_cluster_reprs(a_graph, number_clusters=6)
assignm = [0]*300
for i, cl_rprs in enumerate(cluster_reprs):
    assignm[cl_rprs] = i + 1

plot_nx_graph(a_graph, assignm, assignm)

got node 161 with dist 12 to cluster centers
got node 246 with dist 12 to cluster centers
got node 295 with dist 12 to cluster centers
got node 199 with dist 11 to cluster centers


In [116]:
rs, asgm = multiple_rumor_source_prediction(a_graph, 2)

In [117]:
def visualise_cluster_graph(g: networkx.Graph, rumor_centers: List[List[int]], assignment: List[int]) -> None:
    max_assignment = max(assignment) + 1
    for i, r_cs in enumerate(rumor_centers):
        for r_c in r_cs:
            assignment[r_c] = i + max_assignment

    plot_nx_graph(g, assignment, assignment)

In [119]:
visualise_cluster_graph(a_graph, rs, asgm)

In [120]:
max(asgm)

5

In [1]:
plot_nx_graph(a_graph)

NameError: name 'plot_nx_graph' is not defined